In [1]:
import sys
sys.path.append("../db")

from database import init_db, db_session
from models import Metadata, Output, Parameters

from shapely.geometry import Point
import geopandas as gpd

from osgeo import gdal
from osgeo import gdalconst

from datetime import datetime

init_db()

postgres


In [2]:
admin2 = gpd.read_file('/Users/brandon/Downloads/gadm36_levels_shp/gadm36_2.shp')

In [3]:
admin2['country'] = admin2['NAME_0']
admin2['state'] = admin2['NAME_1']
admin2['admin1'] = admin2['NAME_1']
admin2['admin2'] = admin2['NAME_2']
admin2 = admin2[['geometry','country','state','admin1','admin2']]

In [4]:
def raster2gpd(InRaster,feature_name,nodataval=-9999):
    # open the raster and get some properties
    ds       = gdal.OpenShared(InRaster,gdalconst.GA_ReadOnly)
    GeoTrans = ds.GetGeoTransform()
    ColRange = range(ds.RasterXSize)
    RowRange = range(ds.RasterYSize)
    rBand    = ds.GetRasterBand(1) # first band
    nData    = rBand.GetNoDataValue()
    if nData == None:
        nData = nodataval # set it to something if not set
    else:
        print("NoData value is {0}".format(nData))

    # specify the center offset
    HalfX    = GeoTrans[1] / 2
    HalfY    = GeoTrans[5] / 2

    points = []
    for ThisRow in RowRange:
        RowData = rBand.ReadAsArray(0,ThisRow,ds.RasterXSize,1)[0]
        for ThisCol in ColRange:
            if RowData[ThisCol] != nData:
                
                # TODO: implement filters on valid pixels
                # for example, the below would ensure pixel values are between -100 and 100
                #if (RowData[ThisCol] <= 100) and (RowData[ThisCol] >= -100):

                X = GeoTrans[0] + ( ThisCol * GeoTrans[1] )
                Y = GeoTrans[3] + ( ThisRow * GeoTrans[5] ) # Y is negative so it's a minus
                # this gives the upper left of the cell, offset by half a cell to get centre
                X += HalfX
                Y += HalfY
                points.append([Point(X, Y),X,Y,RowData[ThisCol],feature_name])

    return gpd.GeoDataFrame(points, columns=['geometry','longitude','latitude','feature_value','feature_name'])

In [5]:
InRaster = 'C2P2_LPJmL_yield_backcasts_2018/global_anomalies_maize_LIM_IRRIGATION_LIM_NITROGEN_pctl,95_REFLIM_IRRIGATION_REFLIM_NITROGEN.tif'
feature_name = 'Yield'

In [6]:
gdf = raster2gpd(InRaster,feature_name)

NoData value is 9999.0


Perform geospatial merge:

> Note that the left join will leave in points that do not intersect GADM shapes. Upon inspection, some of these points are just off the coastline and are likely caused by the selection of the center of the pixel, given the large pixel size

In [7]:
gdf = gpd.sjoin(gdf, admin2, how="left", op='intersects')
del(gdf['index_right'])

/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [8]:
gdf['datetime'] = datetime(year=2018, month=1, day=1)
del(gdf['geometry'])

In [9]:
gdf['run_id'] = 'test'
gdf['model'] = 'yield_anomalies_lpjml'

In [10]:
gdf.head()

,longitude,latitude,feature_value,feature_name,country,state,admin1,admin2,datetime,run_id,model
0,42.25,63.25,-48.747810,Yield,Russia,Arkhangel'sk,Arkhangel'sk,Vinogradovskiy rayon,2018-01-01,test,yield_anomalies_lpjml
1,35.25,62.75,-66.384605,Yield,Russia,Karelia,Karelia,Medvezh'egorskiy rayon,2018-01-01,test,yield_anomalies_lpjml
2,36.75,62.75,-65.273956,Yield,Russia,Karelia,Karelia,Pudozhskiy rayon,2018-01-01,test,yield_anomalies_lpjml
3,37.25,62.75,-64.960899,Yield,Russia,Arkhangel'sk,Arkhangel'sk,Onezhskiy rayon,2018-01-01,test,yield_anomalies_lpjml
4,38.25,62.75,-60.795589,Yield,Russia,Arkhangel'sk,Arkhangel'sk,Plesetskiy rayon,2018-01-01,test,yield_anomalies_lpjml


In [11]:
# Generate metadata for "test" run
met = Metadata(run_id='test')
met.run_label = 'Some run trial'
met.model = 'yield_anomalies_lpjml'
met.raw_output_link = 's3://somewhere'
met.run_description = 'testing'
db_session.add(met)
db_session.commit()

In [12]:
# perform bulk insert of entire geopandas DF
db_session.bulk_insert_mappings(Output, gdf.to_dict(orient="records"))
db_session.commit()